In [21]:
import json
import pandas
import requests
import pandas_datareader
import math
import numpy as np
import matplotlib.pyplot as plt
import talib 
import warnings
warnings.filterwarnings("ignore")
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
## sklerrn lib ##
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle

total_oriented = [ # 力道
  'value',  # 價值
  'trend',  # 趨勢
  'swing',  # 波段
  'chip',  # 籌碼
  'dividend' # 股利
]
value_indicators = {
    '營收年增率': 'REVENUE-YOY',
    '營業現金流對稅後淨利比': 'NCFO-NETINC',
    '本益比河流圖': 'PE-RATIO'
}
trend_indicators = ['surfing-trend', 'power-squeeze', 'power-squeeze-momentum']
chip_indicators = {
    '機構持有率': 'inst-ownship',
    '持有機構數': 'inst-count'
}

api_key = 'tv-ffb6c7b0-b70d-480f-b074-e3290cc29287'
url_head = 'https://api.tradingvalley.com/public'
api_url = {
    'symbol_rating': lambda oriented, symbol: f'{url_head}/historical/rating/{oriented}/{symbol}',
    'symbol_score_rating': lambda oriented, symbol, score: f'{url_head}/historical/rating/{oriented}/{symbol}/{score}', 
    'symbol_indicators': lambda oriented, symbol, indicator: f'{url_head}/historical/{oriented}/{symbol}/{indicator}',
    'vix': f'{url_head}/historical/stock/vix'
}


session = requests.Session()
session.headers.update({'X-API-KEY': api_key})
    


In [12]:
def value_average(target_name,start_date,end_date,average_day):
    #print(target_name)
    target = pandas_datareader.data.DataReader(target_name+'.US', 'stooq', start = start_date, end = end_date)
    target.to_csv(f"target.csv")
    target=target.iloc[::-1].reset_index()
    #print(target)
    
    symbol = target_name
    total_ratings = {}
    for oriented in total_oriented:
      url = api_url['symbol_rating'](oriented, symbol)
      #print(f'將呼叫 {url}')
      response = session.get(url)
      total_ratings.update({
          oriented: pandas.DataFrame(json.loads(response.text)['data']).set_index('date', drop = True)['score']})
    total_ratings = pandas.concat(total_ratings, axis = 1, sort = True)
    total_ratings.to_csv(f"target_5.csv")
    #print(total_ratings)
    
    money=100000
    num_stock=0
    value=0
    accumulation=0.0
    average=0.0
    day_profit = np.zeros(target.shape[0])
    assets = np.zeros(target.shape[0])
    day = np.zeros(target.shape[0])
    reverse=0
    for i in range(target.shape[0]-1):
        date_idx=str(target.iat[i,0])
        if pandas.isnull(total_ratings.loc[date_idx[0:10],"value"])==False:
            old_value = value
            value = total_ratings.loc[date_idx[0:10],"value"]
            reverse = value-old_value
            #print(value)
        if i<average_day:
            accumulation = accumulation + target.iat[i,1]
        elif i>=average_day:
            accumulation = accumulation + target.iat[i,1]
            accumulation = accumulation - target.iat[i-average_day,1]
            average = accumulation/average_day
        if value==5:
            if num_stock==0:
                if target.iat[i,1]<=average:
                    num_stock=math.floor(money/target.iat[i,1])
                    money=money-target.iat[i,1]*num_stock
                    #print('buy at ',target.iat[i,1])
                    #print('num_stock = ',num_stock)
                    #print('money = ',money)
        elif reverse<0:
            if(num_stock>0):
                money=money+target.iat[i,1]*num_stock
                num_stock=0
                #print('sell at ',target.iat[i,1])
                #print('money = ',money)
        if num_stock>0:
            day_profit[i]=((target.iat[i,4]-target.iat[i,1])/target.iat[i,1])*100
        else:
            day_profit[i]=0
        assets[i]=money+target.iat[i,1]*num_stock
        day[i]=i
    if(num_stock>0):
        money=money+target.iat[i+1,1]*num_stock
        num_stock=0
        #print('sell at ',target.iat[i+1,1])
        #print('money = ',money)
    porfit=day_profit.mean()
    risk=day_profit.std()
    sharp=porfit/risk*(target.shape[0]**0.5)
    #print('sharp ratio = ',sharp)
    #print('總報酬率 = ',((money-100000)/100000)*100,'%')
    #plt.xlabel('day') 
    #plt.ylabel('assets') 
    #plt.plot(day, assets)
    #plt.show()
    return ((money-100000)/100000)*100,sharp

In [13]:
def value_breakthrough(target_name,start_date,end_date,breakthrough_num):
    #print(target_name)
    target = pandas_datareader.data.DataReader(target_name+'.US', 'stooq', start = start_date, end = end_date)
    target.to_csv(f"target.csv")
    target=target.iloc[::-1].reset_index()
    #print(target)
    
    symbol = target_name
    total_ratings = {}
    for oriented in total_oriented:
      url = api_url['symbol_rating'](oriented, symbol)
      #print(f'將呼叫 {url}')
      response = session.get(url)
      total_ratings.update({
          oriented: pandas.DataFrame(json.loads(response.text)['data']).set_index('date', drop = True)['score']})
    total_ratings = pandas.concat(total_ratings, axis = 1, sort = True)
    total_ratings.to_csv(f"target_5.csv")
    #print(total_ratings)
    
    money=100000
    num_stock=0
    value=0
    breakthrough=0.0
    day_profit = np.zeros(target.shape[0])
    assets = np.zeros(target.shape[0])
    day = np.zeros(target.shape[0])
    reverse=0
    for i in range(target.shape[0]-1):
        date_idx=str(target.iat[i,0])
        if pandas.isnull(total_ratings.loc[date_idx[0:10],"value"])==False:
            old_value = value
            value = total_ratings.loc[date_idx[0:10],"value"]
            reverse = value-old_value
            #print(value)
        if i<1:
            breakthrough = 0
        elif i>1:
            breakthrough = ((target.iat[i,1]-target.iat[i-1,1])/target.iat[i-1,1])*100
        if value==5:
            if num_stock==0:
                if breakthrough>=breakthrough_num:
                    num_stock=math.floor(money/target.iat[i,1])
                    money=money-target.iat[i,1]*num_stock
                    #print('buy at ',target.iat[i,1])
                    #print('num_stock = ',num_stock)
                    #print('money = ',money)
        elif reverse<0:
            if(num_stock>0):
                money=money+target.iat[i,1]*num_stock
                num_stock=0
                #print('sell at ',target.iat[i,1])
                #print('money = ',money)
        if num_stock>0:
            day_profit[i]=((target.iat[i,4]-target.iat[i,1])/target.iat[i,1])*100
        else:
            day_profit[i]=0
        assets[i]=money+target.iat[i,1]*num_stock
        day[i]=i
    if(num_stock>0):
        money=money+target.iat[i+1,1]*num_stock
        num_stock=0
        #print('sell at ',target.iat[i+1,1])
        #print('money = ',money)
    porfit=day_profit.mean()
    risk=day_profit.std()
    sharp=porfit/risk*(target.shape[0]**0.5)
    #print('sharp ratio = ',sharp)
    #print('總報酬率 = ',((money-100000)/100000)*100,'%')
    #plt.xlabel('day') 
    #plt.ylabel('assets') 
    #plt.plot(day, assets)
    #plt.show()
    return ((money-100000)/100000)*100,sharp
    

In [14]:
# 111065513 劉杰 code

## util
def set_label(df):
  label_df = pandas.DataFrame(index=df.index)
  label_df['week_trend'] = np.where(df.Close.shift(-5) > df.Close, 1, 0)
  return label_df

def BRMA(df):
  Close_price = df.Close
  MA5 = Close_price.rolling(window=5).mean()
  MA10 = Close_price.rolling(window=10).mean()
  MA20 = Close_price.rolling(window=20).mean()
  MA60 = Close_price.rolling(window=60).mean()
  df['BRMA5'] = (Close_price - MA5) / MA5
  df['BRMA10'] = (Close_price - MA10) / MA10
  df['BRMA20'] = (Close_price - MA20) / MA20
  df['BRMA60'] = (Close_price - MA60) / MA60
  return df

def RSI(df, period):
  Close_diff = df.Close.diff()
  up = Close_diff.clip(lower=0)
  down = -1 * Close_diff.clip(upper=0)
  # calculate SMA
  ma_up = up.rolling(window=period).mean()
  ma_down = down.rolling(window=period).mean()
  rsi = ma_up / ma_down
  rsi = 100 - (100/(1 + rsi))
  df['rsi'] = rsi
  return df

def MACD(df):
  MA12 = df.Close.ewm(span=12, min_periods=12).mean()
  MA26 = df.Close.ewm(span=26, min_periods=26).mean()
  df["DIF"] = MA12 - MA26
  df['Signal'] = df.DIF.ewm(span=9, min_periods=9).mean()
  df["Histrogram"] = df.DIF - df.Signal
  return df

def get_complete_data(df):
  df = RSI(df, 14)
  df = MACD(df)
  df = BRMA(df)
  df = df.fillna(0)
  df = df[df.columns.difference(['Open', 'High', 'Low', 'Close'])]
  # print(df)
  # print(df)
  return df

# main strategy function
def ML_strategy(target='COST', Test_Start_Date='2018-01-01', Test_End_Date='2022-12-31'):
  # set training period 
  Train_Start_Date = '2008-01-01' # 訓練資料起始日期
  Train_End_Date = '2017-12-31'  # 訓練資料結束日期
  ORIENTED = 'trend'        # 使用指標
  
  # set API
  api_key = 'tv-ffb6c7b0-b70d-480f-b074-e3290cc29287'
  url_head = 'https://api.tradingvalley.com/public'
  api_url = {
      'symbol_rating': lambda oriented, symbol, start, end: f'{url_head}/historical/rating/{oriented}/{symbol}?start_at={start}&end_at={end}',
      'symbol_score_rating': lambda oriented, symbol, score: f'{url_head}/historical/rating/{oriented}/{symbol}/{score}', 
      'symbol_indicators': lambda oriented, symbol, indicator: f'{url_head}/historical/{oriented}/{symbol}/{indicator}',
      'vix': f'{url_head}/historical/stock/vix'
  }

  session = requests.Session()
  session.headers.update({'X-API-KEY': api_key})

  # load training data
  Train_data = pandas_datareader.data.DataReader(target+'.US', 'stooq', start=Train_Start_Date, end=Train_End_Date).iloc[::-1]
  url = api_url['symbol_rating'](ORIENTED, target, Train_Start_Date, Train_End_Date)
  # print(f'將呼叫 {url}')
  response = session.get(url)
  training_swing = pandas.DataFrame(json.loads(response.text)['data'])
  Train_data['score'] = training_swing['score'].to_numpy()
  
  # load testing data
  Test_data = pandas_datareader.data.DataReader(target+'.US', 'stooq', start=Test_Start_Date, end=Test_End_Date).iloc[::-1]
  url = api_url['symbol_rating'](ORIENTED, target, Test_Start_Date, Test_End_Date)
  # print(f'將呼叫 {url}')
  response = session.get(url)
  testing_swing = pandas.DataFrame(json.loads(response.text)['data'])
  Test_data['score'] = testing_swing['score'].to_numpy()

  # set training and testing label
  train_label_df = set_label(Train_data)
  test_label_df = set_label(Test_data)

  # calculate other feature
  X_train = get_complete_data(Train_data)
  X_test = get_complete_data(Test_data)
  
  # model fit and predict
  model = DecisionTreeClassifier(max_depth=6)
  model.fit(X_train, train_label_df)
  prediction = model.predict(X_test)
  result = Test_data
  result['prediction'] = prediction
  # print("ML_Strategy training data score", model.score(X_train, train_label_df))
  # print("ML_Strategy testing data score:", model.score(X_test, test_label_df))

  result['status'] = result.prediction.shift(1).fillna(0)
  # 當model的預測從0->1時，隔天的開盤買進
  result['buy_cost'] = result.Open[np.where((result.status == 1) * (result.status.shift(1) == 0))[0]]
  # 從1->0時，0的那天的開盤賣出
  result['sell_cost'] = result.Open[np.where((result.status == 0) * (result.status.shift(1) == 1))[0]]
  # 把缺值補上0
  test = result.fillna(0)
  buy_cost = np.array(test.buy_cost[test.buy_cost != 0])
  sell_price = np.array(test.sell_cost[test.sell_cost != 0])
  if len(buy_cost) > len(sell_price) :
    buy_cost = buy_cost[:-1]
  trade_return = sell_price / buy_cost - 1

  test['trade_ret'] = 0
  sell_dates = test.sell_cost[test.sell_cost != 0].index
  test.loc[sell_dates, 'trade_ret'] = trade_return

  test['open_ret'] = test.Open / test.Open.shift(1) - 1
  test['strategy_ret'] = test.status.shift(1) * test.open_ret
  test = test.fillna(0)

  test['buy_and_hold_equity'] = (test.open_ret + 1).cumprod()
  test['strategy_equity'] = (test.strategy_ret + 1).cumprod()

  trade_count = len(sell_dates)
  trade_count_per_year = trade_count / (len(test)/252)
  profit_factor = trade_return[trade_return > 0].sum() / abs(trade_return[trade_return < 0].sum())
  mean_net_return = np.mean(trade_return)
  acc_ret = test.strategy_equity[-1] - 1
  strategy_ear = test.strategy_equity[-1] ** (252/len(test)) - 1
  strategy_std = test.strategy_ret.std() * (252 ** 0.5)
  strategy_sharpe = (strategy_ear - 0.01) / strategy_std
  rate_of_return = test.strategy_equity[-1] * 100
  # print("報酬率:%d%%" % (rate_of_return))
  # print("夏普值:", strategy_sharpe)
  # ax = test.strategy_equity.plot()
  # ax.set_ylabel("rate of return")
  return rate_of_return, strategy_sharpe

In [15]:
# 111062524 朱育民 code
def Trend_EMA30(target_name, start_date, end_date):
  All_Price = pandas_datareader.data.DataReader(target_name+'.US', 'stooq', start=start_date, end=end_date).iloc[::-1]
  All_Price['EMA_30'] = All_Price['Close'].ewm(span=30).mean()

  Balance = 100000 
  Share = 0      
  url = f'{url_head}/historical/rating/trend/{target_name}?start_at={start_date}&end_at={end_date}'
  response = session.get(url)
  All_data = pandas.DataFrame(json.loads(response.text)['data'])

  PNL_List = []
  Assets_List = [Balance]
  Date_List = []
  Score_List = []
  Buy_date_list = []
  Sell_date_list = []
  Start_Buying = False

  for idx in reversed(All_data.index):

    date = All_data.loc[idx,'date']
    price_open = All_Price.loc[date,'Open']
    price_close = All_Price.loc[date,'Close']
    score = All_data.loc[idx, 'score']
    Date_List.append(date)
    Score_List.append(score)

    ema30 = All_Price.loc[date, 'EMA_30']

    if Start_Buying == False and score < 3:
      Start_Buying = True

    if (score == 5 or idx == 0) and Share > 0:

      Balance += Share * price_open
      Sell_date_list.append(date)
      Share = 0
      Start_Buying = False

    elif Start_Buying == True and score <= 2 and price_open >= ema30 and Balance >= price_open:

      share_n = int(Balance / price_open)
      Balance -= share_n * price_open
      Share += share_n
      Buy_date_list.append(date)
    
    # Calculate Profit and Loss per day
    today_assets = Share * price_close + Balance
    previous_assets = Assets_List[-1]
    Assets_List.append(today_assets)
    Today_PNL = (today_assets - previous_assets) / previous_assets
    PNL_List.append(Today_PNL)

  Profit = Balance - 100000
  Sharp_Ratio = np.mean(PNL_List) / np.std(PNL_List) * math.sqrt(252)
  #print(f'報酬率: {round((Profit/100000)*100, 2)}%')
  #print('夏普比率:', round(Sharp_Ratio, 3))

  return round((Profit/100000)*100, 2), round(Sharp_Ratio, 3)


In [16]:
# 111064554 鄒汯憲 code
def swing_analysis(target_name,start_date,end_date):

  api_url = {
    'symbol_rating': lambda oriented, symbol, begin, final: 
    f'{url_head}/historical/rating/{oriented}/{symbol}?start_at={begin}&end_at={final}'
  }
  url = api_url['symbol_rating']('swing', target_name, start_date, end_date)

  session = requests.Session()
  session.headers.update({'X-API-KEY': api_key})
  response = session.get(url)

  if response.status_code == 200:
    swing_data = pandas.DataFrame(json.loads(response.text)['data'])
    #print(swing_data)

  stock_data = pandas_datareader.data.DataReader(target_name, 'stooq', start = start_date, end = end_date)
  #print(stock_data)


  ##投資模擬

  asset = 100000
  #print(f'波段力道分數分析(標的為{obj}，回測從{start}到{end}):')
  #print('\n若投資人的總資產為', asset)

  stock_num = 0
  List_total_asset = []

  for i in range(len(stock_data)):
    if swing_data.iat[len(swing_data)-1-i,1] >= 3: #進場時機
      if asset > 0:
        if asset >= stock_data.iat[len(stock_data)-1-i,0]: #開盤時買入
          x = int(asset / stock_data.iat[len(stock_data)-1-i,0])
          asset -= x * stock_data.iat[len(stock_data)-1-i,0]
          stock_num += x
          #print('在',swing_data.iat[len(swing_data)-1-i,0],'時，買入股票，剩餘資產為',asset)
          #print('在',swing_data.iat[len(swing_data)-1-i,0],'時，買入股票，剩餘股票張數為',stock_num)
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          record = i

        else:
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          continue

      elif stock_num > 0:
        List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
        continue

      else:
        print('bankrupt !')
        break

    elif swing_data.iat[len(swing_data)-1-i,1] < 3: #出場時機
      if stock_num > 0:                     #兩天開盤價股票的價差
        diff = stock_num * (stock_data.iat[len(stock_data)-1-i,0] - stock_data.iat[len(stock_data)-1-record,0])
        asset += stock_num * stock_data.iat[len(stock_data)-1-record,0] + diff
        stock_num = 0
        #print('在',swing_data.iat[len(swing_data)-1-i,0],'時，賣出股票，剩餘資產為',asset)
        #print('在',swing_data.iat[len(swing_data)-1-i,0],'時，賣出股票，剩餘股票張數為',stock_num)
        List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)

      else:
        if asset == 0 and stock_num == 0:
          print('bankrupt !')
          break

        else:
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          continue

    else:
      List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
      continue

  #print('====================================================================')
  stock_profit = stock_num * stock_data.iat[0,3] #最後一天收盤時全數賣出
  maximum_profit = stock_profit + asset
  total_profit_ratio = 100 * ((maximum_profit - 100000) / 100000)

  R = stock_data.cumsum()
  r = (R - R.shift(1))/R.shift(1)
  sr = r.mean()/r.std() * np.sqrt(252)

  #print('到最後一天收盤時，將股票全賣，最大收益為', maximum_profit)
  #print('總投資報酬率為', round(total_profit_ratio,2), '%')
  #print('sharpe ratio =', round(sr[3],6))
  return round(total_profit_ratio,2), round(sr[3],6)

def trend_analysis(target_name,start_date,end_date):

  api_url = {
    'symbol_rating': lambda oriented, symbol, begin, final: 
    f'{url_head}/historical/rating/{oriented}/{symbol}?start_at={begin}&end_at={final}'
  }
  url = api_url['symbol_rating']('trend', target_name, start_date, end_date)

  session = requests.Session()
  session.headers.update({'X-API-KEY': api_key})
  response = session.get(url)

  if response.status_code == 200:
    trend_data = pandas.DataFrame(json.loads(response.text)['data'])
    #print(trend_data)

  stock_data = pandas_datareader.data.DataReader(target_name, 'stooq', start = start_date, end = end_date)
  #print(stock_data)


  ##投資模擬

  asset = 100000
  #print(f'趨勢力道分數分析(標的為{obj}，回測從{start}到{end}):')
  #print('\n若投資人的總資產為', asset)

  stock_num = 0
  List_total_asset = []

  for i in range(len(stock_data)):
    if trend_data.iat[len(trend_data)-1-i,1] >= 4: #進場時機
      if asset > 0:
        if asset >= stock_data.iat[len(stock_data)-1-i,0]: #開盤時買入
          x = int(asset / stock_data.iat[len(stock_data)-1-i,0])
          asset -= x * stock_data.iat[len(stock_data)-1-i,0]
          stock_num += x
          #print('在',trend_data.iat[len(trend_data)-1-i,0],'時，買入股票，剩餘資產為',asset)
          #print('在',trend_data.iat[len(trend_data)-1-i,0],'時，買入股票，剩餘股票張數為',stock_num)
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          record = i

        else:
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          continue

      elif stock_num > 0:
        List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
        continue

      else:
        print('bankrupt !')
        break

    elif trend_data.iat[len(trend_data)-1-i,1] < 4: #出場時機
      if stock_num > 0:                     #兩天開盤價股票的價差
        diff = stock_num * (stock_data.iat[len(stock_data)-1-i,0] - stock_data.iat[len(stock_data)-1-record,0])
        asset += stock_num * stock_data.iat[len(stock_data)-1-record,0] + diff
        stock_num = 0
        #print('在',trend_data.iat[len(trend_data)-1-i,0],'時，賣出股票，剩餘資產為',asset)
        #print('在',trend_data.iat[len(trend_data)-1-i,0],'時，賣出股票，剩餘股票張數為',stock_num)
        List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)

      else:
        if asset == 0 and stock_num == 0:
          print('bankrupt !')
          break

        else:
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          continue

    else:
      List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
      continue

  #print('====================================================================')
  stock_profit = stock_num * stock_data.iat[0,3] #最後一天收盤時全數賣出
  maximum_profit = stock_profit + asset
  total_profit_ratio = 100 * ((maximum_profit - 100000) / 100000)

  R = stock_data.cumsum()
  r = (R - R.shift(1))/R.shift(1)
  sr = r.mean()/r.std() * np.sqrt(252)

  #print('到最後一天收盤時，將股票全賣，最大收益為', maximum_profit)
  #print('總投資報酬率為', round(total_profit_ratio,2), '%')
  #print('sharpe ratio =', round(sr[3],6))
  return round(total_profit_ratio,2), round(sr[3],6)

def surfing_trend_analysis(target_name,start_date,end_date):

  api_url = {
    'symbol_indicators': lambda oriented, symbol, indicator, begin, final: 
    f'{url_head}/historical/{oriented}/{symbol}/{indicator}?start_at={begin}&end_at={final}'
  }
  url = api_url['symbol_indicators']('trend', target_name, 'surfing-trend', start_date, end_date)

  session = requests.Session()
  session.headers.update({'X-API-KEY': api_key})
  response = session.get(url)

  if response.status_code == 200:
    surfing_trend = pandas.DataFrame(json.loads(response.text)['data'])
    #print(surfing_trend)

  stock_data = pandas_datareader.data.DataReader(target_name, 'stooq', start = start_date, end = end_date)
  #print(stock_data)


  ##投資模擬

  asset = 100000
  #print(f'趨勢因子surfing-trend分析(標的為{obj}，回測從{start}到{end}):')
  #print('\n若投資人的總資產為', asset)

  stock_num = 0
  List_total_asset = []

  for i in range(len(stock_data)):
    if surfing_trend.iat[len(surfing_trend)-1-i,1] >= 0: #進場時機
      if asset > 0:
        if asset >= stock_data.iat[len(stock_data)-1-i,0]: #開盤時買入
          x = int(asset / stock_data.iat[len(stock_data)-1-i,0])
          asset -= x * stock_data.iat[len(stock_data)-1-i,0]
          stock_num += x
          #print('在',surfing_trend.iat[len(surfing_trend)-1-i,0],'時，買入股票，剩餘資產為',asset)
          #print('在',surfing_trend.iat[len(surfing_trend)-1-i,0],'時，買入股票，剩餘股票張數為',stock_num)
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          record = i

        else:
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          continue

      elif stock_num > 0:
        List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
        continue

      else:
        print('bankrupt !')
        break

    elif surfing_trend.iat[len(surfing_trend)-1-i,1] < 0: #出場時機
      if stock_num > 0:                     #兩天開盤價股票的價差
        diff = stock_num * (stock_data.iat[len(stock_data)-1-i,0] - stock_data.iat[len(stock_data)-1-record,0])
        asset += stock_num * stock_data.iat[len(stock_data)-1-record,0] + diff
        stock_num = 0
        #print('在',surfing_trend.iat[len(surfing_trend)-1-i,0],'時，賣出股票，剩餘資產為',asset)
        #print('在',surfing_trend.iat[len(surfing_trend)-1-i,0],'時，賣出股票，剩餘股票張數為',stock_num)
        List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)

      else:
        if asset == 0 and stock_num == 0:
          print('bankrupt !')
          break

        else:
          List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
          continue

    else:
      List_total_asset.append(asset+stock_data.iat[len(stock_data)-1-i,0]*stock_num)
      continue

  #print('====================================================================')
  stock_profit = stock_num * stock_data.iat[0,3] #最後一天收盤時全數賣出
  maximum_profit = stock_profit + asset
  total_profit_ratio = 100 * ((maximum_profit - 100000) / 100000)

  R = stock_data.cumsum()
  r = (R - R.shift(1))/R.shift(1)
  sr = r.mean()/r.std() * np.sqrt(252)

  #print('到最後一天收盤時，將股票全賣，最大收益為', maximum_profit)
  #print('總投資報酬率為', round(total_profit_ratio,2), '%')
  #print('sharpe ratio =', round(sr[3],6))
  return round(total_profit_ratio,2), round(sr[3],6)

In [17]:
#110065519施詠舜
#KD
class KDCross(Strategy):
    def init(self):
        self.slowk, self.slowd = self.I(talib.STOCH, self.data.High,self.data.Low,
                        self.data.Close, fastk_period=9, slowk_period=3, slowd_period=3)

    def next(self):
        if crossover(self.slowk, self.slowd):
            self.buy()
        elif crossover(self.slowd, self.slowk):
            self.sell()


def strategy_KD(a , b, c):
  format = "%Y-%m-%d"
  date_start = datetime.strptime(b, format)
  date_end = datetime.strptime(c, format)
  df = pandas_datareader.data.DataReader(a, 'stooq', date_start, date_end)
  #print('\nStochastic Oscillator(KD)')
  bt_KD = Backtest(data=df, strategy=KDCross, cash=10000, commission=0.000, exclusive_orders=True)
  output = bt_KD.run()
  d = output['Return [%]']
  e = output['Sharpe Ratio']
  return d, e

#EMA
class ema_cross(Strategy):
    span1 = 10
    span2 = 20

    def init(self):
        Close1 = self.data.Close
        self.ma1 = self.I(func=self.ema, values=Close1, n=self.span1)
        self.ma2 = self.I(func=self.ema, values=Close1, n=self.span2)

    def next(self):
        # If mal crosses above ma2 , buy the asset
        if crossover(series1=self.ma1, series2=self.ma2):
            self.buy()
        elif crossover(series1=self.ma2, series2=self.ma1):
            self.sell()

    def ema(self, values, n):
        return pandas.Series(values).ewm(span=n).mean()


def strategy_EMA(a , b, c):
  format = "%Y-%m-%d"
  date_start = datetime.strptime(b, format)
  date_end = datetime.strptime(c, format)
  df = pandas_datareader.data.DataReader(a, 'stooq', date_start, date_end)
  #print('\nExponential Moving Average Cross')
  bt_ema_cross = Backtest(data=df, strategy=ema_cross, cash=10000, commission=0.000)
  output = bt_ema_cross.run(span1=10, span2=20)
  d = output['Return [%]']
  e = output['Sharpe Ratio']
  return d, e

#RSI
def rsi(array, n):
  gain = pandas.Series(array).diff()
  loss = gain.copy()
  gain[gain < 0] = 0
  loss[loss > 0] = 0
  rs = gain.rolling(n).mean() / loss.abs().rolling(n).mean()
  return 100 - 100 / (1 + rs)

class rsi_simple(Strategy):
  ww_rsi = 14
  oversold_level = 30
  overbought_level = 70
  def init(self):
    Close1 = self.data.Close
    self.rsi_14 = self.I(rsi, Close1, self.ww_rsi)
  def next(self):
    if self.rsi_14 < self.oversold_level:
      self.buy()
    elif self.rsi_14 > self.overbought_level:
      self.sell()

def strategy_RSI(a , b, c):
  format = "%Y-%m-%d"
  date_start = datetime.strptime(b, format)
  date_end = datetime.strptime(c, format)
  df = pandas_datareader.data.DataReader(a, 'stooq', date_start, date_end)
  #print('\nRelative Strength Index')
  bt_rsi = Backtest(df, rsi_simple, cash=10000, commission=0.000)
  output = bt_rsi.run()
  d = output['Return [%]']
  e = output['Sharpe Ratio']
  return d, e

In [18]:
def find_best_strategy(target_name,start_date,end_date):
    max_gain=0
    max_SR=0
    best_gain_method=0
    best_SR_method=0
    gain,SR=value_average(target_name,start_date,end_date,30)
    if gain>max_gain:
        max_gain=gain
        best_gain_method=0
    if SR>max_SR:
        max_SR=SR
        best_SR_method=0
    gain,SR=value_average(target_name,start_date,end_date,90)
    if gain>max_gain:
        max_gain=gain
        best_gain_method=1
    if SR>max_SR:
        max_SR=SR
        best_SR_method=1
    gain,SR=value_breakthrough(target_name,start_date,end_date,3)
    if gain>max_gain:
        max_gain=gain
        best_gain_method=2
    if SR>max_SR:
        max_SR=SR
        best_SR_method=2
    gain,SR=value_breakthrough(target_name,start_date,end_date,5)
    if gain>max_gain:
        max_gain=gain
        best_gain_method=3
    if SR>max_SR:
        max_SR=SR
        best_SR_method=3
    gain,SR=ML_strategy(target_name, start_date, end_date)   # 111065513劉杰 method
    if gain>max_gain:
        max_gain=gain
        best_gain_method=4
    if SR>max_SR:
        max_SR=SR
        best_SR_method=4
    gain,SR=Trend_EMA30(target_name, start_date, end_date)   # 111062524 朱育民 method
    if gain>max_gain:
        max_gain=gain
        best_gain_method=5
    if SR>max_SR:
        max_SR=SR
        best_SR_method=5
    gain,SR=swing_analysis(target_name, start_date, end_date)   # 111064554 鄒汯憲 method
    if gain>max_gain:
        max_gain=gain
        best_gain_method=6
    if SR>max_SR:
        max_SR=SR
        best_SR_method=6
    gain,SR=trend_analysis(target_name, start_date, end_date)   # 111064554 鄒汯憲 method
    if gain>max_gain:
        max_gain=gain
        best_gain_method=7
    if SR>max_SR:
        max_SR=SR
        best_SR_method=7
    gain,SR=surfing_trend_analysis(target_name, start_date, end_date)   # 111064554 鄒汯憲 method
    if gain>max_gain:
        max_gain=gain
        best_gain_method=8
    if SR>max_SR:
        max_SR=SR
        best_SR_method=8
    gain,SR=strategy_KD(target_name, start_date, end_date)   # 111065519 施詠舜 method
    if gain>max_gain:
        max_gain=gain
        best_gain_method=9
    if SR>max_SR:
        max_SR=SR
        best_SR_method=9
    gain,SR=strategy_EMA(target_name, start_date, end_date)   # 111065519 施詠舜 method
    if gain>max_gain:
        max_gain=gain
        best_gain_method=10
    if SR>max_SR:
        max_SR=SR
        best_SR_method=10
    gain,SR=strategy_RSI(target_name, start_date, end_date)   # 111065519 施詠舜 method
    if gain>max_gain:
        max_gain=gain
        best_gain_method=11
    if SR>max_SR:
        max_SR=SR
        best_SR_method=11

    print('對於此標的的最佳總收益策略: 策略',best_gain_method,'，其回測總報酬率 = ',max_gain,'%')
    strategy_discribe(best_gain_method)
    
    print('對於此標的的最佳夏普值策略: 策略',best_SR_method,'，其回測夏普值 = ',max_SR)
    strategy_discribe(best_SR_method)

In [19]:
def strategy_discribe(strategy):
    if strategy==0:
        print('策略0描述:當五力基本面分數到5且價格低於月均價時買入，而基本面分數反轉時賣出')
    if strategy==1:
        print('策略1描述:當五力基本面分數到5且價格低於季均價時買入，而基本面分數反轉時賣出')
    if strategy==2:
        print('策略2描述:當五力基本面分數到5且股價突破（日漲幅3%以上）時買入，而基本面分數反轉時賣出')
    if strategy==3:
        print('策略3描述:當五力基本面分數到5且股價突破（日漲幅5%以上）時買入，而基本面分數反轉時賣出')
    if strategy==4:
        print('策略4描述:當ML模型預測從0轉1時買入，當模型預測從1轉0時賣出')
    if strategy==5:
        print('策略5描述:當五力的趨勢分數低於2且股價突破月均價時買入，當趨勢分數到5時賣出')
    if strategy==6:
        print('策略6描述:當五力的波段力道分數到3以上時買進，到2以下時賣出')
    if strategy==7:
        print('策略7描述:當五力的趨勢力道分數到4以上時買進，到3以下時賣出')
    if strategy==8:
        print('策略8描述:當五力的趨勢力道因子surfing-trend由負轉正時買進，由正轉負時賣出')
    if strategy==9:
        print('策略9描述:Stochastic Oscillator(KD指標)，當slowk漲破slod時買入，當slowd漲破slok時賣出')
    if strategy==10:
        print('策略10描述:Exponential Moving Average(EMA)，當ma1漲破ma2時買入，當ma2漲破ma1時賣出')        
    if strategy==11:
        print('策略11描述:RSI相對強弱指標，當RSI值低於30時買進，當RSI值高於70時賣出')         

In [22]:
#value_average(name,start_date,end_date,90)
target_name=input('輸入標的名稱:')
start_date=input('輸入回測起始日:')
end_date=input('輸入回測結束日:')
find_best_strategy(target_name,start_date,end_date)

輸入標的名稱:AAPL
輸入回測起始日:2020-01-01
輸入回測結束日:2022-01-01
對於此標的的最佳總收益策略: 策略 8 ，其回測總報酬率 =  231.58 %
策略8描述:當五力的趨勢力道因子surfing-trend由負轉正時買進，由正轉負時賣出
對於此標的的最佳夏普值策略: 策略 6 ，其回測夏普值 =  3.609741
策略6描述:當五力的波段力道分數到3以上時買進，到2以下時賣出
